## Key Parameters:

3 components (representing 3 market regimes)
Fixed random seed (42) for reproducible results

Input Features for BTC+DXY Model:
When btc_only=False, it uses 4 features:

BTC_ret - Bitcoin returns
DXY_ret - US Dollar Index returns
BTC_vol - Bitcoin volatility (4-period rolling standard deviation)
DXY_vol - DXY volatility (4-period rolling standard deviation)

## Preprocessing:

StandardScaler - Normalizes all features to have mean=0, std=1
This is crucial for GMM since it assumes features are on similar scales

## Training Process:

Fit GMM on the 4 standardized features
Predict regimes (0, 1, or 2) for each time period
Dynamic labeling - Maps regime numbers to meaningful names based on average Bitcoin returns:

Lowest avg BTC return → "Risk-Off"
Middle avg BTC return → "Neutral"
Highest avg BTC return → "Risk-On"

## Walk-Forward Implementation:

Retrains every 3 months with expanding window of data
Uses all historical data up to current period for training
Prevents look-ahead bias by never using future data

## Reason for GMM?
Unsupervised - No need for labeled training data
Probabilistic - Can handle overlapping market conditions
Flexible - Can capture complex, non-linear relationships between BTC and DXY
Regime-based - Naturally segments market into distinct behavioral periods

The model essentially learns to identify when Bitcoin and the Dollar Index are in different "relationship modes" and trades accordingly.

# The Model

In [1]:
import warnings
warnings.filterwarnings('ignore')

Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from dateutil.relativedelta import relativedelta
import gdown

## Helper Functions

In [3]:
def load_data(btc_path, dxy_path):
    btc_df = pd.read_csv(btc_path)
    dxy_df = pd.read_csv(dxy_path)

    btc_df['Date'] = pd.to_datetime(btc_df['Date'])
    btc_df['Price'] = btc_df['Price'].astype(str).str.replace(',', '').str.replace('$', '')
    btc_df['Price'] = pd.to_numeric(btc_df['Price'], errors='coerce')
    btc_df = btc_df.dropna(subset=['Price']).sort_values('Date')

    dxy_df['Date'] = pd.to_datetime(dxy_df['Date'])
    dxy_df['Price'] = pd.to_numeric(dxy_df['Price'], errors='coerce')
    dxy_df = dxy_df.dropna(subset=['Price']).sort_values('Date')

    print(f"Original DXY data: {len(dxy_df)} days")
    print(f"Bitcoin date range: {len(btc_df)} days")
    print(f"DXY date range: {dxy_df['Date'].min()} to {dxy_df['Date'].max()}")
    print(f"BTC date range: {btc_df['Date'].min()} to {btc_df['Date'].max()}")

    # Check if we have overlapping data
    overlap_start = max(btc_df['Date'].min(), dxy_df['Date'].min())
    overlap_end = min(btc_df['Date'].max(), dxy_df['Date'].max())

    if overlap_start > overlap_end:
        print("ERROR: No overlapping dates between BTC and DXY data!")
        return pd.DataFrame()

    print(f"Overlapping period: {overlap_start} to {overlap_end}")

    # Filter both datasets to overlapping period only
    btc_overlap = btc_df[(btc_df['Date'] >= overlap_start) & (btc_df['Date'] <= overlap_end)]
    dxy_overlap = dxy_df[(dxy_df['Date'] >= overlap_start) & (dxy_df['Date'] <= overlap_end)]

    print(f"BTC data in overlap: {len(btc_overlap)} days")
    print(f"DXY data in overlap: {len(dxy_overlap)} days")

    # Create complete date range for overlapping period
    date_range = pd.date_range(start=overlap_start, end=overlap_end, freq='D')

    # Reindex DXY to include all dates and interpolate
    dxy_indexed = dxy_overlap.set_index('Date')
    dxy_complete = dxy_indexed.reindex(date_range)

    # Fix deprecation warning - use ffill() and bfill()
    dxy_complete['Price'] = dxy_complete['Price'].interpolate(method='linear')
    dxy_complete['Price'] = dxy_complete['Price'].ffill().bfill()

    # Reset index properly
    dxy_complete = dxy_complete.reset_index()
    dxy_complete = dxy_complete.rename(columns={'index': 'Date', 'Price': 'DXY'})

    # Merge with BTC data
    merged = pd.merge(
        btc_overlap[['Date', 'Price']].rename(columns={'Price': 'BTC'}),
        dxy_complete[['Date', 'DXY']],
        on='Date',
        how='inner'
    )

    print(f"Final merged dataset: {len(merged)} days")
    return merged.dropna()

In [4]:
    # Compute returns and volatility
def compute_features(df):
    df['BTC_ret'] = df['BTC'].pct_change()
    df['DXY_ret'] = df['DXY'].pct_change()
    df['BTC_vol'] = df['BTC_ret'].rolling(4).std()
    df['DXY_vol'] = df['DXY_ret'].rolling(4).std()
    df['volume'] = df['BTC'].rolling(2).mean()
    return df.dropna()

In [5]:
# Dynamic regime labeling
def dynamic_regime_labels(train_df, regimes):
    df = train_df.copy()
    df['Regime'] = regimes
    means = df.groupby('Regime')['BTC_ret'].mean()
    sorted_regimes = means.sort_values()
    regime_summary = pd.DataFrame({
        'Regime': sorted_regimes.index,
        'Mean_Return': sorted_regimes.values,
        'Date': train_df['Date'].max()
    })
    labels = {sorted_regimes.index[0]: 'Risk-Off',
              sorted_regimes.index[1]: 'Neutral',
              sorted_regimes.index[2]: 'Risk-On'}
    return labels, regime_summary

In [6]:
# Apply trained model
def apply_model(test_df, gmm, scaler, regime_labels, btc_only=False):
    if btc_only:
        features = test_df[['BTC_ret', 'BTC_vol']].values
    else:
        features = test_df[['BTC_ret', 'DXY_ret', 'BTC_vol', 'DXY_vol']].values
    features_scaled = scaler.transform(features)
    regimes = gmm.predict(features_scaled)
    test_df['Regime'] = regimes
    test_df['Regime_Label'] = test_df['Regime'].map(regime_labels)
    return test_df

    

In [7]:
# GMM regime model
def train_regime_model(train_df, btc_only=False):
    if btc_only:
        features = train_df[['BTC_ret', 'BTC_vol']].values
    else:
        features = train_df[['BTC_ret', 'DXY_ret', 'BTC_vol', 'DXY_vol']].values
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    gmm = GaussianMixture(n_components=3, random_state=42)
    regimes = gmm.fit_predict(features_scaled)
    regime_labels, regime_summary = dynamic_regime_labels(train_df, regimes)
    return gmm, scaler, regime_labels, regime_summary

In [8]:
# Walkforward GMM regime model - FIXED VERSION
def walkforward_markov(test_df, train_df, btc_only=False, freq_months=3):
    output = []
    regime_summaries = []
    initial_train_end = train_df['Date'].max()
    test_start = test_df['Date'].min()
    cursor = test_start
    end_date = test_df['Date'].max()
    while cursor < end_date:
        test_end = cursor + relativedelta(months=freq_months)
        current_train = pd.concat([
            train_df[train_df['Date'] < cursor],
            test_df[test_df['Date'] < cursor]
        ]).drop_duplicates().sort_values('Date')
        current_test = test_df[(test_df['Date'] >= cursor) & (test_df['Date'] < test_end)]
        if len(current_train) < 20 or len(current_test) < 1:
            cursor += relativedelta(months=freq_months)
            continue
        gmm, scaler, regime_labels, regime_summary = train_regime_model(current_train, btc_only)
        regime_summaries.append(regime_summary)
        labeled = apply_model(current_test.copy(), gmm, scaler, regime_labels, btc_only)
        output.append(labeled)
        cursor += relativedelta(months=freq_months)
    final_df = pd.concat(output).sort_values('Date') if output else test_df
    all_summaries = pd.concat(regime_summaries).reset_index(drop=True)
    return final_df, gmm, all_summaries

In [9]:
# FIXED: Simulate strategy with proper capital tracking
def simulate_strategy(df, strategy_column, budget=100000, max_trades=300, fee=0.001):
    df = df.copy()  # Work on a copy to avoid modifying original
    trades = 0
    capital = budget
    capital_history = []
    for idx, row in df.iterrows():
        r = row[strategy_column]
        if strategy_column not in row or pd.isna(r):
            capital_history.append(capital)
            continue
        if trades >= max_trades:
            capital_history.append(capital)
            continue
        if abs(r) < 0.01:  # Skip small movements
            capital_history.append(capital)
            continue
        # Apply return with fees (2x fee for round-trip)
        #capital *= (1 + r) * (1 - 2 * fee)
        position_size = 1  # Only use 50% of capital per trade
        position_return = r * position_size
        capital += capital * position_return * (1 - 2 * fee)
        trades += 1
        capital_history.append(capital)
    # Ensure we have the right number of capital values
    if len(capital_history) != len(df):
        print(f"Warning: Length mismatch - capital_history: {len(capital_history)}, df: {len(df)}")
        # Pad or truncate as needed
        while len(capital_history) < len(df):
            capital_history.append(capital_history[-1] if capital_history else budget)
        capital_history = capital_history[:len(df)]
    df[strategy_column + '_Capital'] = capital_history
    # Create trade log for reference
    trade_log = [(i, capital_history[i]) for i in range(len(capital_history)) if i == 0 or capital_history[i] != capital_history[i-1]]
    return df, trade_log

In [10]:
# Metrics
def get_metrics(df, capital_column):
    capital = df[capital_column]
    returns = capital.pct_change().dropna()
    sharpe = returns.mean() / returns.std() * np.sqrt(365) if returns.std() > 0 else 0
    drawdown = (capital / capital.cummax()) - 1
    max_dd = drawdown.min()
    total_return = (capital.iloc[-1] / capital.iloc[0]) - 1
    return sharpe, max_dd, total_return
def compare_regime_models(ml_dxy, ml_btc):
    """
    Compare regime classifications between DXY+BTC and BTC-only models
    """
    print("\n" + "="*60)
    print("REGIME COMPARISON: DXY+BTC vs BTC-ONLY")
    print("="*60)
    dxy_btc_regimes = ml_dxy['Regime_Label'].value_counts()
    btc_only_regimes = ml_btc['Regime_Label'].value_counts()
    comparison_df = pd.DataFrame({
        'DXY+BTC Model': dxy_btc_regimes,
        'BTC-Only Model': btc_only_regimes
    }).fillna(0)
    comparison_df['DXY+BTC %'] = (comparison_df['DXY+BTC Model'] / comparison_df['DXY+BTC Model'].sum()) * 100
    comparison_df['BTC-Only %'] = (comparison_df['BTC-Only Model'] / comparison_df['BTC-Only Model'].sum()) * 100
    print("\nRegime Distribution Comparison:")
    print(comparison_df.round(1))
    # Analyze regime agreement
    regime_agreement = ml_dxy['Regime_Label'] == ml_btc['Regime_Label']
    agreement_rate = regime_agreement.mean() * 100
    print(f"\nModel Agreement: {agreement_rate:.1f}% of periods have same regime classification")
    if agreement_rate < 70:
        print("Low agreement suggests DXY data significantly changes regime identification")
    elif agreement_rate > 85:
        print("High agreement suggests DXY adds minimal regime information")
    else:
        print("Moderate agreement suggests DXY provides useful but not dominant regime signals")

## Download and Prepare Data

In [11]:
# File IDs from shared Google Drive files
btc_train_id = '1HUjG7uls0IM5vMHhvbstNzo8gYJrGb5m'
dxy_train_id = '15XwX1v55VyO9DOzXmXvUwQp26m4tf64h'
btc_test_id  = '1FDPzI0BkEGBGukFhXqHy_O0Sa7jzx4Uq'
dxy_test_id  = '1ULkN0o_wNmqQA5U8rlwF-Jt7yOSUo9Fj'

In [12]:
# Download files into Colab local environment
gdown.download(f'https://drive.google.com/uc?id={btc_train_id}', 'btc_train.csv', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={dxy_train_id}', 'dxy_train.csv', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={btc_test_id}', 'btc_test.csv', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={dxy_test_id}', 'dxy_test.csv', quiet=False)

train_path_btc = 'btc_train.csv'
train_path_dxy = 'dxy_train.csv'
test_path_btc = 'btc_test.csv'
test_path_dxy = 'dxy_test.csv'

Downloading...
From: https://drive.google.com/uc?id=1HUjG7uls0IM5vMHhvbstNzo8gYJrGb5m
To: /Users/nichomac/Documents/ml/btc_train.csv
100%|███████████████████████████████████████████████████████████████████| 3.97k/3.97k [00:00<00:00, 5.01MB/s]
Downloading...
From: https://drive.google.com/uc?id=15XwX1v55VyO9DOzXmXvUwQp26m4tf64h
To: /Users/nichomac/Documents/ml/dxy_train.csv
100%|███████████████████████████████████████████████████████████████████| 2.99k/2.99k [00:00<00:00, 7.84MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FDPzI0BkEGBGukFhXqHy_O0Sa7jzx4Uq
To: /Users/nichomac/Documents/ml/btc_test.csv
100%|███████████████████████████████████████████████████████████████████| 95.6k/95.6k [00:00<00:00, 1.22MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ULkN0o_wNmqQA5U8rlwF-Jt7yOSUo9Fj
To: /Users/nichomac/Documents/ml/dxy_test.csv
100%|███████████████████████████████████████████████████████████████████| 54.2k/54.2k [00:00<00:00, 1.05MB/s]


In [13]:
# Load and prepare data
train_df = load_data(train_path_btc, train_path_dxy)
test_df = load_data(test_path_btc, test_path_dxy)
train_df = compute_features(train_df)
test_df = compute_features(test_df)

Original DXY data: 52 days
Bitcoin date range: 52 days
DXY date range: 2021-01-03 00:00:00 to 2021-12-26 00:00:00
BTC date range: 2021-01-03 00:00:00 to 2021-12-26 00:00:00
Overlapping period: 2021-01-03 00:00:00 to 2021-12-26 00:00:00
BTC data in overlap: 52 days
DXY data in overlap: 52 days
Final merged dataset: 52 days
Original DXY data: 901 days
Bitcoin date range: 1272 days
DXY date range: 2022-01-03 00:00:00 to 2025-06-25 00:00:00
BTC date range: 2022-01-01 00:00:00 to 2025-06-25 00:00:00
Overlapping period: 2022-01-03 00:00:00 to 2025-06-25 00:00:00
BTC data in overlap: 1270 days
DXY data in overlap: 901 days
Final merged dataset: 1270 days


In [14]:
def classify_three_regimes(df, dxy_col='DXY'):
    dxy_mean = df[dxy_col].mean()
    dxy_std = df[dxy_col].std()

    # Optimized thresholds for balanced distribution
    risk_on_threshold = dxy_mean - 0.4 * dxy_std  #
    risk_off_threshold = dxy_mean + 0.4 * dxy_std  #

    def get_regime(dxy_val):
        if dxy_val < risk_on_threshold:
            return 'Risk-On'
        elif dxy_val > risk_off_threshold:
            return 'Risk-Off'
        else:
            return 'Neutral'

    df['Regime_Label'] = df[dxy_col].apply(get_regime)
    return df, risk_on_threshold, risk_off_threshold

test_df, r_on_thresh, r_off_thresh = classify_three_regimes(test_df)


print(f"Data loaded - Train: {len(train_df)} samples, Test: {len(test_df)} samples")
print(f"Test period: {test_df['Date'].min()} to {test_df['Date'].max()}")

Data loaded - Train: 48 samples, Test: 1266 samples
Test period: 2022-01-07 00:00:00 to 2025-06-25 00:00:00


Strategies

In [15]:
# Strategy 1 & 2: ML with walk-forward (DXY+BTC and BTC-only)
global ml_dxy, gmm_dxy, regime_boundaries_dxy
ml_dxy, gmm_dxy, regime_boundaries_dxy = walkforward_markov(test_df.copy(), train_df.copy(), btc_only=False, freq_months=3)

global ml_btc, gmm_btc, regime_boundaries_btc
ml_btc, gmm_btc, regime_boundaries_btc = walkforward_markov(test_df.copy(), train_df.copy(), btc_only=True, freq_months=3)

In [16]:
def run_models(b):
    global ml_dxy, ml_btc  # must come before you read from ml_dxy

    ml_dxy.reset_index(drop=False, inplace=True)
    ml_btc.reset_index(drop=False, inplace=True)

    print('rerunning models')
    
    # Ensure all strategies cover same time period for fair comparison
    common_start = max(
        ml_dxy['Date'].min(),
        ml_btc['Date'].min()
    )
    common_end = min(
        ml_dxy['Date'].max(),
        ml_btc['Date'].max()
    )
    
    print(f"Common evaluation period: {common_start} to {common_end}")
    
    # Filter to common period
    ml_dxy = ml_dxy[(ml_dxy['Date'] >= common_start) & (ml_dxy['Date'] <= common_end)]
    ml_btc = ml_btc[(ml_btc['Date'] >= common_start) & (ml_btc['Date'] <= common_end)]
    
    ml_weights_dxy = {'Risk-On': ronslider.value, 'Neutral': 0.0, 'Risk-Off': roffslider.value}
    ml_weights_btc = {'Risk-On': ronslider.value, 'Neutral':0.0, 'Risk-Off': roffslider.value}
    
    ml_dxy['ML_Strategy_DXY'] = ml_dxy['Regime_Label'].map(ml_weights_dxy) * ml_dxy['BTC_ret']
    ml_btc['ML_Strategy_BTC'] = ml_btc['Regime_Label'].map(ml_weights_btc) * ml_btc['BTC_ret']
    
    # Simulate all strategies
    ml_dxy, _ = simulate_strategy(ml_dxy, 'ML_Strategy_DXY')
    ml_btc, _ = simulate_strategy(ml_btc, 'ML_Strategy_BTC')

    ml_dxy = ml_dxy.set_index('Date').sort_index()
    ml_btc = ml_btc.set_index('Date').sort_index()

    try:
        update_plot()
    except NameError:
        pass  # or log a message

In [17]:
import ipywidgets as widgets
from IPython.display import display

ronslider = widgets.FloatSlider(description="Risk-On",
    value=0.5,        # default value
    min=0.0,          # lower bound
    max=1.0,          # upper bound)
)
roffslider = widgets.FloatSlider(description="Risk-Off",
    value=-0.25,        # default value
    min=-1.0,          # lower bound
    max=0.0,          # upper bound
)

rerun = widgets.Button(description='Rerun Models',
                       disabled=False,
                       tooltip='Rerun Models',
                       icon='check')
rerun.on_click(run_models)

In [18]:
run_models(rerun)

rerunning models
Common evaluation period: 2022-01-07 00:00:00 to 2025-06-25 00:00:00


# Visualization

In [21]:
import plotly.graph_objs as go
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import clear_output

# Set the correct renderer
pio.renderers.default = 'notebook_connected'  # fallback to 'notebook' if this fails

# Widgets
date_slider = widgets.IntSlider(value=0, min=0, max=len(ml_dxy)-100, step=1, description='Index')
step_button = widgets.Button(
    description='Step Forward',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Step Forward Three Months',
    icon='forward' # (FontAwesome names without the `fa-` prefix)
)
time_value = [ml_dxy.index.min()];
def button_click(b):
    time_value[0] += relativedelta(months=3)
    new_pos = ml_dxy.index.get_loc(time_value[0])
    date_slider.value = new_pos
    update_plot()
step_button.on_click(button_click)
model_selector = widgets.Dropdown(
    options=['DXY+BTC', 'BTC'],
    value='DXY+BTC',
    description='Model Choice:',
    disabled=False,
)
BTC_DXY_selector = widgets.ToggleButtons(
    options=['BTC', 'DXY', 'None'],
    description='Extra Plot:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)


# Output area for the figure
out = widgets.Output()

def update_plot(change=None):
    with out:

        selected = model_selector.value
        
        if selected == 'DXY+BTC':
            df = ml_dxy
            strategy = 'ML_Strategy_DXY_Capital'
        else:
            df = ml_btc
            strategy = 'ML_Strategy_BTC_Capital'
            
        clear_output(wait=True)
        
        start_idx = time_value[0]
        window_size = relativedelta(months=3)
        end_date = start_idx + relativedelta(months=3)
        
        df_slice = df.loc[df.index.min():end_date].reset_index()
        
        fig = go.Figure()
        
        # Regime to color map
        colors = {'Risk-On': '#00D37E', 'Neutral': '#DAA520', 'Risk-Off': '#FF685F'}
        added_legends = set()

        if BTC_DXY_selector.value == 'BTC':
            fig.add_trace(go.Scatter(
                x=df_slice['Date'], 
                y=df_slice['BTC'],
                mode='lines',
                name='BTC',
                yaxis='y2',
                line=dict(color='#FD9540')
            ))
            y2_title = 'BTC'
            y2_tickformat='$,.2f'
        elif BTC_DXY_selector.value == 'DXY':
            fig.add_trace(go.Scatter(
                x=df_slice['Date'], 
                y=df_slice['DXY'],
                mode='lines',
                name='DXY',
                yaxis='y2',
                line=dict(color='#00beff')
            ))
            y2_title='DXY'
            y2_tickformat=''

        i_tracker = 0  # track position of regime start
        currenttracker = df_slice.iloc[i_tracker]
        
        df_slice = df_slice.copy()
        df_slice['Regime_Change'] = (df_slice['Regime_Label'] != df_slice['Regime_Label'].shift()).cumsum()
        
        # We'll use this to avoid duplicate legends
        added_legends = set()
        
        # To track overlap at boundaries
        prev_segment_end = None
        
        for _, segment in df_slice.groupby('Regime_Change'):
            regime = segment['Regime_Label'].iloc[0]
            show_legend = regime not in added_legends
        
            # If there's an overlap point from the previous segment, prepend it
            if prev_segment_end is not None:
                segment = pd.concat([prev_segment_end, segment])
        
            # Save last row to use as overlap with the next segment
            prev_segment_end = segment.tail(1)
        
            fig.add_trace(go.Scatter(
                x=segment['Date'],
                y=segment[strategy],
                mode='lines+markers',
                name=regime,
                marker=dict(size=10),
                line=dict(color=colors.get(regime, 'gray'), width=4),
                customdata=segment[['BTC_ret', 'Regime_Label']].values,
                hoverlabel=dict(
                    bgcolor="white",
                    font_size=14,
                    font_family="ABC Whyte Unlicensed Trial",
                    font_color="black"
                ),
                hovertemplate=
                    '<b>Date:</b> %{x|%Y-%m-%d}<br>' +
                    '<b>Capital:</b> $%{y:,.2f}<br>' +
                    '<b>Strategy:</b> %{customdata[1]}<br>' +
                    '<b>BTC Return:</b> %{customdata[0]:.2%}<br>' +
                    '<extra></extra>',
                showlegend=show_legend
            ))
        
            added_legends.add(regime)

        
            added_legends.add(regime)

            time_value[0] = ml_dxy.index[date_slider.value]


        fig.update_layout(
            title='Monk Mode',
            xaxis_title='Date',
            yaxis=dict(title='Capital', tickformat='$,.2f', side='left'),
            height=500,
            legend=dict(x=0.02, y=1, xanchor='left', yanchor='top',
                bgcolor='rgba(255,255,255,0.5)',  # semi-transparent background
                bordercolor='black',
                borderwidth=1
            ),
            font=dict(family='ABC Whyte Unlicensed Trial', size=16),
            template='plotly_white'
        )
        y2_title = y2_title if 'y2_title' in locals() else None
        if y2_title:
            fig.layout['yaxis2'] = dict(
                title=y2_title,
                tickformat=y2_tickformat,
                overlaying='y',
                side='right',
                showgrid=False
            )
        fig.show(config={'scrollZoom': True})

# Trigger updates
for w in [date_slider, step_button, model_selector, BTC_DXY_selector]:
    w.observe(update_plot, names='value')

    

hbox1 = widgets.HBox([date_slider, step_button])
hbox1.layout.border = '0px solid transparent'
hbox1.layout.border_bottom = '3px solid #eee'
hbox1.layout.padding = '0 0 10px 0'  # Just bottom padding

hbox2 = widgets.HBox([roffslider, ronslider, rerun])
hbox2.layout.border = '0px solid transparent'
hbox2.layout.border_bottom = '3px solid #eee'
hbox2.layout.padding = '0 0 10px 0'  # Just bottom padding

# No border on the last item
vbox = widgets.VBox([hbox1, hbox2], 
layout=widgets.Layout(
    gap='30px',
    padding='30px',
    height='200px',
    justify_content='space-around'
))

display(widgets.HBox([model_selector, BTC_DXY_selector], layout=widgets.Layout(width='800px',
    justify_content='space-between')), out, vbox)

update_plot()

Output()